# EDA-Calidad del dataset Gas_Emissions

## Librerías

In [1]:
import os
import warnings

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Ajustes

Se ignoran los warnings.

In [3]:
warnings.filterwarnings("ignore")

Se modifica la configuración para que las tablas salgan completas en las salidas.

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)

## Importacion del dataset

Ruta actual.

In [5]:
ruta_actual = os.getcwd()

ruta_actual

'c:\\Users\\franc\\Desktop\\taxi_electrico_nyc\\EDA'

Ruta raíz del proyecto.

In [6]:
ruta_del_proyecto = os.path.dirname(
    ruta_actual
    )

ruta_del_proyecto

'c:\\Users\\franc\\Desktop\\taxi_electrico_nyc'

Ruta del dataset.

In [7]:
ruta_del_dataset = os.path.join(
    ruta_del_proyecto, 
    'assets', 
    'NYC_Greenhouse_Gas_Emissions_Inventory_20241206.csv'
    )

ruta_del_dataset

'c:\\Users\\franc\\Desktop\\taxi_electrico_nyc\\assets\\NYC_Greenhouse_Gas_Emissions_Inventory_20241206.csv'

Se importa el dataset.

In [8]:
gas_emissions = pd.read_csv(
    ruta_del_dataset
    )

## Exploración General

Primera fila.

In [12]:
gas_emissions.head(1)

,Inventory Type,Sectors Sector,Category Full,Category Label,Source Full,Source Label,Source Units,CY 2005 Consumed,CY 2005 tCO2e (100 yr GWP),CY 2005 Source MMBtu,CY 2005 tCO2e (20 yr GWP),CY 2006 Consumed,CY 2006 tCO2e (100 yr GWP),CY 2006 Source MMBtu,CY 2006 tCO2e (20 yr GWP),CY 2007 Consumed,CY 2007 tCO2e (100 yr GWP),CY 2007 Source MMBtu,CY 2007 tCO2e (20 yr GWP),CY 2008 Consumed,CY 2008 tCO2e (100 yr GWP),CY 2008 Source MMBtu,CY 2008 tCO2e (20 yr GWP),CY 2009 Consumed,CY 2009 tCO2e (100 yr GWP),CY 2009 Source MMBtu,CY 2009 tCO2e (20 yr GWP),CY 2010 Consumed,CY 2010 tCO2e (100 yr GWP),CY 2010 Source MMBtu,CY 2010 tCO2e (20 yr GWP),CY 2011 Consumed,CY 2011 tCO2e (100 yr GWP),CY 2011 Source MMBtu,CY 2011 tCO2e (20 yr GWP),CY 2012 Consumed,CY 2012 tCO2e (100 yr GWP),CY 2012 Source MMBtu,CY 2012 tCO2e (20 yr GWP),CY 2013 Consumed,CY 2013 tCO2e (100 yr GWP),CY 2013 Source MMBtu,CY 2013 tCO2e (20 yr GWP),CY 2014 Consumed,CY 2014 tCO2e (100 yr GWP),CY 2014 Source MMBtu,CY 2014 tCO2e (20 yr GWP),CY 2015 Consumed,CY 2015 tCO2e (100 yr GWP),CY 2015 Source MMBtu,CY 2015 tCO2e (20 yr GWP),CY 2016 Consumed,CY 2016 tCO2e (100 yr GWP),CY 2016 Source MMBtu,CY 2016 tCO2e (20 yr GWP),CY 2017 Consumed,CY 2017 tCO2e (100 yr GWP),CY 2017 Source MMBtu,CY 2017 tCO2e (20 yr GWP),CY 2018 Consumed,CY 2018 tCO2e (100 yr GWP),CY 2018 Source MMBtu,CY 2018 tCO2e (20 yr GWP),CY 2019 Consumed,CY 2019 tCO2e (100 yr GWP),CY 2019 Source MMBtu,CY 2019 tCO2e (20 yr GWP),CY 2020 Consumed,CY 2020 tCO2e (100 yr GWP),CY 2020 Source MMBtu,CY 2020 tCO2e (20 yr GWP),CY 2021 Consumed,CY 2021 tCO2e (100 yr GWP),CY 2021 Source MMBtu,CY 2021 tCO2e (20 yr GWP),CY 2022 Consumed,CY 2022 tCO2e (100 yr GWP),CY 2022 Source MMBtu,CY 2022 tCO2e (20 yr GWP),CY 2023 Consumed,CY 2023 tCO2e (100 yr GWP),CY 2023 Source MMBtu,CY 2023 tCO2e (20 yr GWP),2005 - 2022 % change Consumed,2005 - 2022 % change tCO2e,2005 - 2022 % change Source MMBtu,2005 - 2022 % change tCO2e 20 yr GWP
0,GPC,Stationary Energy,Residential (small and large residential),Residential,#2 fuel oil,#2 fuel oil,gallon,1.466460e+08,1517233.2,20439518.9,1549279.75,1.236520e+08,1279331.85,17234613.31,1306353.52,1.422401e+08,1471649.29,19825431.82,1502733.03,1.311070e+08,1356463.27,18273694.95,1385114.09,146483621.5,1515553.24,20416887.16,1547564.31,1.411770e+08,1460649.45,19677246.7,1491500.85,1.381669e+08,1429506.28,19257699.26,1459699.89,1.252074e+08,1295424.21,17451402.73,1322785.78,1.603248e+08,1658757.6,22346075.33,1693793.39,1.229829e+08,1272409.02,17141351.95,1299284.47,160363033.9,1659152.83,22351399.66,1694196.97,1.225838e+08,1268280.5,17085734.23,1295068.74,1.057673e+08,1094293.32,14741853.15,1117406.66,1.344716e+08,1391274.0,18742650.26,1420660.08,136255620.5,1409731.99,18991308.38,1439507.93,97015873.57,1003748.54,13522072.46,1024949.42,94453737.87,977240.1,13164961.98,997881.07,1.059489e+08,1096171.18,14767150.78,1119324.18,90526881.46,936611.94,12617636.74,956394.78,-38.268426,-38.268426,-38.268426,-38.268426


Tamaño.

In [15]:
print("Cantidad de filas: ")
print(gas_emissions.shape[0])
print()
print("Cantidad de columnas:")
print(gas_emissions.shape[1])

Cantidad de filas: 
85

Cantidad de columnas:
87


## Tipos de Datos

Nombre de cada columna y tipo de dato.

In [16]:
gas_emissions.dtypes

Inventory Type                           object
Sectors Sector                           object
Category Full                            object
Category Label                           object
Source Full                              object
Source Label                             object
Source Units                             object
CY 2005 Consumed                        float64
CY 2005 tCO2e (100 yr GWP)              float64
CY 2005 Source MMBtu                    float64
CY 2005 tCO2e (20 yr GWP)               float64
CY 2006 Consumed                        float64
CY 2006 tCO2e (100 yr GWP)              float64
CY 2006 Source MMBtu                    float64
CY 2006 tCO2e (20 yr GWP)               float64
CY 2007 Consumed                        float64
CY 2007 tCO2e (100 yr GWP)              float64
CY 2007 Source MMBtu                    float64
CY 2007 tCO2e (20 yr GWP)               float64
CY 2008 Consumed                        float64
CY 2008 tCO2e (100 yr GWP)              

## Valores Faltantes

Valores Cadena Vacía en las columnas de tipo object.

In [17]:
columnas_object = gas_emissions.select_dtypes(include='object').columns

columnas_object

Index(['Inventory Type', 'Sectors Sector', 'Category Full', 'Category Label',
       'Source Full', 'Source Label', 'Source Units'],
      dtype='object')

In [19]:
filas_con_valores_vacios = gas_emissions[
    columnas_object].apply(lambda x: x == "").any(axis=1)

print('Cantidad de valores faltantes:')
print(filas_con_valores_vacios.sum())

Cantidad de valores faltantes:
0


## Valores Nulos

In [29]:
print('Cantidad de valores nulos')

print(gas_emissions.isnull().sum().sum())

Cantidad de valores nulos
0


El dataset tiene todos sus valores completos.

## Valores Duplicados

In [ ]:
print(gas_emissions.duplicated().sum())

0


No hay ningun valor duplicado.

## Diccionario de datos

In [48]:
ruta_del_diccionario = os.path.join(
    ruta_del_proyecto, 
    'assets', 
    'Inventory_of_New_York_City_Greenhouse_Gas_Emissions_Data_Dictionary.xlsx'
    )

ruta_del_diccionario

'c:\\Users\\franc\\Desktop\\taxi_electrico_nyc\\assets\\Inventory_of_New_York_City_Greenhouse_Gas_Emissions_Data_Dictionary.xlsx'

In [49]:
diccionario = pd.read_excel(
    ruta_del_diccionario, 
    sheet_name='Column Information', 
    header=1
    )

In [50]:
diccionario

,Column Name,Column Description,Expected/Allowed Values,Field Limitations,Additional Notes
0,Name of the column exactly as it appears in the dataset.,"A brief, plain-language explanation of what the data in the column means.","Specifies if there is an expected range and/or format of possible values. For example, if the data type is Date & Time, this field will note whether the timestamp is MM/DD/YYYY or MM/YYYY. If the Column Name is ice cream, this field might note that values can be Chocolate, Vanilla or Strawberry.\n\nIf relevant, this field specifies the unit of measurement of the data field, e.g. thousands, millions, $ value, miles, feet, year, etc.","Describes any unique characteristics or potential analytical limitations presented by this field, including: \n- the reasoning for any null, zero, or empty values in the data\n- if the data in the column was integrated from another dataset or organization\n- if the data covered includes a different time period\n- the source of the column and how the data in the column was generated. \n\nFor example, information on how the data in this column was generated can include whether the data was self-reported directly by a person, system generated by a database or agency system, derived through analytical manipulation of other fields or records; or obtained from a different agency.","Provides any additional relevant information about the data in the column, including: \n- definitions of acronyms, special term or codes, or jargon that appears in the field values;\n- the meaning of confusing or non-intuitive values in the data; \n- how the information in this column relates to information in other columns;\n- other unique details about this column."
1,Inventory,Indicator of the inventory associated with that data.,Plain Text,NaN,NaN
2,Sector,Describes the main sectors of the inventory,Plain Text,NaN,NaN
3,Category Full,The full description of main categories of data within each sector,Plain Text,NaN,NaN
4,Category Label,Shorthand description of the main categories of data within each sector,Plain Text,NaN,NaN
5,Source Full,Full description of the energy sources of emissions,Plain Text,NaN,NaN
6,Source Label,Shorthand description of the energy sources of emissions,Plain Text,NaN,NaN
7,Source Units,Units that track the amount of energy sources of emissions,Plain Text,NaN,NaN
8,CY (2005-2023) Consumed,Amount of energy in source units consumed for that category for that CY,Number,NaN,NaN
9,CY (2005-2023) tCO2e (100 yr GWP),Amount of CO2e in tons emitted for that category for that CY,Number,NaN,tCO2e stands for tones (t) of carbon dioxide (CO2) equivalent.
